<a href="https://colab.research.google.com/github/EddyBautista-93/Intro-To-AI-Agents/blob/main/Intro_to_ai_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro to AI Agents - EventForge

For this workshop we will be creating a AI Agent with the ability to help us plan for a event we will be hosting. \
Below is the following tools we will be looking building together.

Tool # | Name | What it adds
--- | --- | ---
1 | DuckDuckGoSearch | Venue Discovery - Built In
2 | Image Generation | Create Promo Visual - Community Tool
3 | Post Gen | Generate Post
4 | Budget Estimator | Financial planning for headcount, food, venue, and swag - Custom


# Getting started

We’ll begin by installing the dependencies needed for building our smart event planning agent.

In [1]:
! pip install smolagents gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.2/108.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 64.7 MB/s eta 0:00:00


The Installs


*   **smolagents**: A lightweight framework for building LLM-powered agents with tool calling support.
*   **gradio**: A UI library for interactive web demos in notebooks



To use models like Mixtral from Hugging Face, you’ll need to authenticate:

In [2]:
from huggingface_hub import notebook_login

notebook_login()


To create a smart AI agent with SmolAgents, you need two core things:


*   A model – the brain of the agent that generates and understands text
*   A set of tools – external functions that the model can call to extend its capabilities

## The Model
Text generation models power your agent. Think of the model as the engine that drives everything forward. SmolAgents supports multiple model types depending on how you want to run your agent.

### Model Options
**TransformerModel** \
Runs a locally initialized transformers pipeline. Ideal for models you have downloaded or finetuned.

**HFApiModel**(What we'll use) \
Uses Hugging Face Inference API via huggingface_hub.InferenceClient to call hosted models. Think LLaMA and Mixtral

**LiteLLM** \
Supports calling 100+ LLMs through the LiteLLM router-userful for OpenAI, claude, etc with unified configs.

**AzureOpenAiServerModel** \
Connects to OpenAI models deployed through Microsoft Azure

**MLXModel** \
Creates a mlx-lm pipeline that run inference on your local machine.


# Tools
Tools give your AI agent ***agency*** - they let the model take actions beyond just talking.\
Tools are passed in like this:
``` python
tools=[]
```
These tools can include things like:


*   Search Engines
*   Image Generators
*   Custom Tools
*   RAG (Retrieval Augmented Generation ) Functions

## Creating the Agent
Once you have the model and tools, you can create a agent like this:

``` python
model_id = "meta-llama/Llama-3.3-70B-Instruct"
model = HfApiModel(model_id=model_id) # By default if you don't pass in a model id the hfapi it uses the qwen model
agent = ToolCallingAgent(tools=[], model=model)
```

# ToolCallingAgent

The ToolCallingAgent is a lightweight class designed to let large language models interact with tools.

Since we're building an agent that plans events, we'll focus on this agent type to give the model the power to search the web, generate images, generate post and estimate budgets.

Key Features of ToolCallingAgent
- **Contextual Memory**: Maintains a chat history, allowing for ongoing reasoning and continuity across multiple prompts..
- **Multi-Step Reasoning**: ToolCallingAgents follow a "think-act-observe" cycle as part of the ReAct framework.
- **Customizability**: You can define:
  * A list of tools for the agent to use.
  * Specify prompt templates.
  * Adjust other parameters like planning intervals during initialization



In short: ToolCallingAgent is what gives your AI brain the power to do more than talk—it can take action.

## Tool 1: DuckDuckGoSearch + Location
The first tool we're using is built directly into the SmolAgents framework: DuckDuckGoSearchTool.

This tools allows the agent to search the web in real time and retrieve up-to-date information. Perfect for finding venues, vendors, and other real-world event planning needs.

This tool:
* Searches DuckDuckGo for relevant results.
* Works out of the box - no API key needed.
* Can be combined with custom tools (like a location preset)

Allow the Agent to Search for the web

In [4]:
from smolagents import ToolCallingAgent, HfApiModel , DuckDuckGoSearchTool, tool # we introduce a built in tool

# You can choose to not pass any model_id to HfApiModel to use a default free
# model Qwen/Qwen2.5-Coder-32B-Instruct
model = HfApiModel()

# Initialize the built in search tool
web_search = DuckDuckGoSearchTool()

# Custom tool that returns the location you are located ( We use be using
# location logic in production)
@tool
def get_location() -> str:
    """
    Returns the default location for event planning.
    """
    return "San Antonio, Texas"

And then you bring it all together!

In [6]:
agent = ToolCallingAgent(tools=[web_search, get_location], model=model)
agent.run("Whats a good indoor venue for a tech focused event with free wifi in my city?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Whats a good indoor venue for a tech focused event with free wifi in my city?                                   │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'get_location' with arguments: {}                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: San Antonio, Texas

[Step 1: Duration 2.24 seconds| Input tokens: 1,236 | Output tokens: 14]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'Based on the search results, several venues in San     │
│ Antonio, Texas offer free Wi-Fi and are suitable for tech-focused events. Some notable options include the      │
│ Witte Museum, Pearl Studio, and San Antonio Glasshouse. These venues provide modern amenities, including        │
│ technology infrastructure and free internet access, making them ideal choices for hosting events.'}             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: Based on the search results, several venues in San Antonio, Texas offer free Wi-Fi and are suitable 
for tech-focused events. Some notable options include the Witte Museum, Pearl Studio, and San Antonio Glasshouse. 
These venues provide modern amenities, including technology infrastructure and free internet access, making them 
ideal choices for hosting events.

[Step 3: Duration 7.88 seconds| Input tokens: 5,043 | Output tokens: 123]

'Based on the search results, several venues in San Antonio, Texas offer free Wi-Fi and are suitable for tech-focused events. Some notable options include the Witte Museum, Pearl Studio, and San Antonio Glasshouse. These venues provide modern amenities, including technology infrastructure and free internet access, making them ideal choices for hosting events.'

## Tool 2: Image Generation
Now that agent can search for venues, let's give it the ability to generate promo visuals for the event.

SmolAgents supports loading third-party tools like text-to-image generators. In this example, we use the open-source model hosted at m-ric/text-to-image, which supports prompts like “minimal line art” or “AI poster design.
This tool:
* Generates images from text prompts
* Hosted on hugging face - no setup or api needed.

In [ ]:
from smolagents import ToolCallingAgent, HfApiModel , DuckDuckGoSearchTool , load_tool
image_generation_tool = load_tool("m-ric/text-to-image", trust_remote_code=True)


In [ ]:
agent = ToolCallingAgent(tools=[image_generation_tool], model=model)
agent.run("Provide minimal line art promo for my event, the theme is automating boring things with AI Agents.")

## Tool 3: LinkedIn Post Generation

With an event idea, venue, and promo visuals in place, its time to get the word out! This tool helps your AI agent write a professional Linkedin Post to promote the event.

Instead of hardcoding a poat, this tool uses the LLM to generate a natural, dynamic social content based on structured inputs.

This tool:
* Generates a LinkedIn ready promo post.
* Uses the same model to write the content via an inernal ToolCallingAgent
* Accepts strucuturd fields like event name, date, and highlight

In [9]:
from smolagents import ToolCallingAgent, HfApiModel, tool

# Initialize the model
model = HfApiModel(model="mistralai/Mixtral-8x7B-Instruct-v0.1")

# Define the post generator tool
@tool
def generate_post(event_name: str, date: str, location: str, highlight: str = "networking and learning") -> str:
    """
    Generates a LinkedIn post for a tech event.

    Args:
        event_name: The name of the event.
        date: The date of the event.
        location: The city or venue where the event is held.
        highlight: What's special about the event. Defaults to 'networking and learning'.
    """
    sub_agent = ToolCallingAgent(model=model, tools=[])
    prompt = (
        f"Write a LinkedIn post promoting an event.\n\n"
        f"Event Name: {event_name}\n"
        f"Date: {date}\n"
        f"Location: {location}\n"
        f"Highlight: {highlight}\n\n"
        "Make it sound professional, friendly, and engaging. Add relevant hashtags."
    )
    return sub_agent.run(prompt)

In [11]:
agent = ToolCallingAgent(tools=[generate_post], model=model)

response = agent.run("Create a LinkedIn post for an event called AI Agent showcase at Geekdom on May the 10th. This event wraps up the ai workshops and talks to showcase what everyone built for the month")
print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Create a LinkedIn post for an event called AI Agent showcase at Geekdom on May the 10th. This event wraps up    │
│ the ai workshops and talks to showcase what everyone built for the month                                        │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'generate_post' with arguments: {'highlight': 'wraps up the ai workshops and talks to showcase    │
│ what everyone built for the month', 'event_name': 'AI Agent showcase', 'date': 'May the 10th', 'location':      │
│ 'Geekdom'}                                                                                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Write a LinkedIn post promoting an event.                                                                       │
│                                                                                                                 │
│ Event Name: AI Agent showcase                                                                                   │
│ Date: May the 10th                                                                                              │
│ Location: Geekdom                                                                                               │
│ Highlight: wraps up the ai workshops and talks to showcase what everyone built for the month                    │
│                                                                                                                 │
│ Make it sound professional, friendly, and engaging. Add relevant hashtags.                                      │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': "🚀 Excitement Awaits at the AI Agent Showcase!         │
│ 🚀\\nJoin us on May 10th at Geekdom for a special event that culminates our month-long AI workshops and         │
│ insightful talks. This showcase will be an opportunity to witness and celebrate the amazing projects that our   │
│ participants have built. Don't miss out on this chance to see the future of AI!\\n📅 Date: May 10th, 2023 \\📍  │
│ Location: Geekdom \\🌐 Theme: Wrapping up AI workshops & Talks Showcase \\[Let's innovate together!]\\"}        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: 🚀 Excitement Awaits at the AI Agent Showcase! 🚀\nJoin us on May 10th at Geekdom for a special event
that culminates our month-long AI workshops and insightful talks. This showcase will be an opportunity to witness 
and celebrate the amazing projects that our participants have built. Don't miss out on this chance to see the 
future of AI!\n📅 Date: May 10th, 2023 \📍 Location: Geekdom \🌐 Theme: Wrapping up AI workshops & Talks Showcase 
\[Let's innovate together!]\

[Step 2: Duration 9.60 seconds| Input tokens: 2,669 | Output tokens: 316]

🚀 Excitement Awaits at the AI Agent Showcase! 🚀\nJoin us on May 10th at Geekdom for a special event that culminates our month-long AI workshops and insightful talks. This showcase will be an opportunity to witness and celebrate the amazing projects that our participants have built. Don't miss out on this chance to see the future of AI!\n📅 Date: May 10th, 2023 \📍 Location: Geekdom \🌐 Theme: Wrapping up AI workshops & Talks Showcase \[Let's innovate together!]\


## Tool 4: Estimate Budget
Let’s add a tool to help estimate the total cost of your event based on the number of attendees and what services you include—like catering, venue rental, and swag.

This lets the agent do basic financial planning in seconds.

This tool:
* Estimates total cost for catering, vene, and swag

In [17]:
from smolagents import ToolCallingAgent, HfApiModel, tool


@tool
def estimate_budget(guests: int,location: str = "San Antonio, Texas", catering: bool = True, venue: bool = True, swag: bool = False) -> dict:
    """
    This tool estimates the total cost of an event.

    Args:
        guests: The number of guests attending the event.
        location: City/region of the event.
        catering: Whether catering is included. Defaults to True.
        venue: Whether a venue is included. Defaults to True.
        swag: Whether swag is included. Defaults to False.
    """
    base_cost = 0
    if catering:
        base_cost += guests * 15  # $15 per guest for food
    if venue:
        base_cost += 500  # Flat rate for venue
    if swag:
        base_cost += guests * 5  # $5 per guest for merch
    return {
        "Location:": location,
        "Estimated Total": f"${base_cost}",
        "Breakdown": {
            "Catering": f"${guests * 15 if catering else 0}",
            "Venue": "$200" if venue else "$0",
            "Swag": f"${guests * 5 if swag else 0}"
        }
    }

In [18]:
agent = ToolCallingAgent(tools=[estimate_budget], model=model)
agent.run("How much will 50 guest cost me for this event and the breakdown of the expenses.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ How much will 50 guest cost me for this event and the breakdown of the expenses.                                │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'estimate_budget' with arguments: {'guests': 50}                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: {'Location:': 'San Antonio, Texas', 'Estimated Total': '$1250', 'Breakdown': {'Catering': '$750', 
'Venue': '$200', 'Swag': '$0'}}

[Step 1: Duration 0.12 seconds| Input tokens: 1,357 | Output tokens: 20]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'The total cost for 50 guests is $1250. The breakdown   │
│ of the expenses is as follows: Catering - $750, Venue - $200, Swag - $0.'}                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: The total cost for 50 guests is $1250. The breakdown of the expenses is as follows: Catering - $750, 
Venue - $200, Swag - $0.

[Step 2: Duration 3.10 seconds| Input tokens: 2,820 | Output tokens: 82]

'The total cost for 50 guests is $1250. The breakdown of the expenses is as follows: Catering - $750, Venue - $200, Swag - $0.'

# Exercise: Custom Tools
Let’s put your creativity to work!

In this section, you’ll write your own custom tools that your agent can use. These boilerplate functions serve as a template—just edit the logic and descriptions to suit your needs.

Example Idea:
* A guest list lookup tool
* A reminder generator
* A checklist builder
* A Spotify playlist suggester for the event

In [19]:
from smolagents import tool

@tool
def custom_tool_1() -> str:
    """
    This tool is a placeholder for your custom functionality.
    """
    return "This tool does nothing yet. Make it do something cool!"

@tool
def custom_tool_2() -> str:
    """
    This tool is another placeholder—get creative with it!
    """
    return "This tool also does nothing... for now."


## Bringing it all together

In [20]:
import gradio as gr
from PIL import Image
import requests
from io import BytesIO

# Tool sets you define elsewhere
tool_sets = {
    "Custom Tools": [custom_tool_1, custom_tool_2],
    "Event Planning": [web_search, estimate_budget, get_location],
    "Post + Image": [generate_post, image_generation_tool],
    # Add more sets as needed
}

# Agent runner function that returns both text and image (if available)
def run_agent(prompt, tool_choice):
    tools = tool_sets.get(tool_choice, [])
    agent = ToolCallingAgent(model=model, tools=tools)
    response = agent.run(prompt)

    # Try to extract image if the tool response is an image URL
    image = None
    if isinstance(response, str) and "http" in response and response.endswith((".jpg", ".png", ".jpeg")):
        try:
            img_data = requests.get(response).content
            image = Image.open(BytesIO(img_data))
        except:
            image = None  # Image fetch failed or not present

    return response, image

# Launch Gradio app
gr.Interface(
    fn=run_agent,
    inputs=[
        gr.Textbox(label="Your Event Planning Prompt", placeholder="e.g. Generate promo image for an AI meetup"),
        gr.Radio(choices=list(tool_sets.keys()), label="Choose Tool Set", value="Post + Image")
    ],
    outputs=[
        gr.Textbox(label="Agent Response"),
        gr.Image(label="Generated Image")
    ],
    title="🧠 Event Planner AI Agent",
    description="Explore how an AI agent becomes smarter and more helpful as you add new tools!"
).launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f916dbf437e98a46e3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
